In [ ]:
%load_ext autoreload
%autoreload 2

from train import *
from models import *
from tools import *

from keras import optimizers
from keras.models import load_model
from keras.utils import to_categorical
from keras.callbacks import *
from keras.preprocessing.image import ImageDataGenerator

from PIL import Image
from IPython.display import display

import numpy as np
import os

In [ ]:
X_train, y_train = load_data()
X_val, y_val = load_data("val")
X_test, _ = load_data("test")

input_shape = X_train[0].shape
print("Input shape: " + str(input_shape))
print(X_train[0][0][0])

y_train = to_categorical(y_train, 200)
y_val = to_categorical(y_val, 200)

In [ ]:
labels = get_label_dict()
words = get_word_labels()

test_images = os.listdir("data/tiny-imagenet-200/test/images/")
assert len(X_test) == len(test_images)

In [ ]:
# Image preprocessing for the data
datagen = ImageDataGenerator(horizontal_flip=True, 
                             shear_range=20., 
                             zoom_range=0.4, 
                             width_shift_range=0.2,
                             height_shift_range=0.2, 
                             channel_shift_range=0.1, 
                             fill_mode="reflect", 
                             preprocessing_function=shuffle_channels)

testgen = ImageDataGenerator()

data = {"train": (X_train, y_train), 
        "val": (X_val, y_val)}

In [ ]:
keras.backend.clear_session()

model = BravoNet(input_shape, "v3", net34=True)
model_prefix = "models/" + model.name + "/" + model.name
model_path = model_prefix + ".h5"

In [ ]:
# Saves the best model based on monitor value between epochs
checkpoint = ModelCheckpoint(model_path, 
                             monitor="val_accuracy", 
                             verbose=1, 
                             save_best_only=True, 
                             period=1)

# Stops training early when no improvement for monitor value
early_stop = EarlyStopping(monitor="val_accuracy", 
                           min_delta=0, 
                           patience=10, 
                           verbose=1)

# Reduces learning rate when monitor value plateaus
reduce_lr = ReduceLROnPlateau(monitor="val_accuracy", 
                              factor=0.316, 
                              patience=5, 
                              verbose=1)

# Logs training data to CSV
restore = False # Whether or not we want to restore weights and continue training from a previous session
csv_log = CSVLogger(model_prefix + ".csv", separator=',', append=restore)

cb_list = [checkpoint, early_stop, reduce_lr, csv_log]

In [ ]:
optim = optimizers.Adam(learning_rate=0.001)

history = train(model_path=model_path, 
                restore=restore, 
                epochs=300,
                model=model, 
                optim=optim, 
                datagen=datagen, 
                testgen=testgen, 
                data=data, 
                cb_list=cb_list, 
                batch_size=32)

In [ ]:
show_history(history)

In [ ]:
# This loads the model and makes predictions on the test set
custom_metrics = {
    "top3_accuracy": top3_acc,
    "top5_accuracy": top5_acc
}
model = load_model(model_path, custom_objects=custom_metrics)
output = model.predict(X_test)
results = []

for result in output:
    results.append(labels[np.argmax(result)])
    
assert len(results) == len(X_test)
print("Test images predicted:", len(results))

In [ ]:
# This writes the test set predictions to file
with open(model_prefix + " Predictions.txt", "w") as test_file:
    for i in range(len(results)):
        test_file.write(test_images[i] + " " + results[i] + "\n")
    print("Predictions saved at", test_file.name)

In [ ]:
# This shows some test set images and the model's predictions on them
display_dim = 128
for _ in range(10):
    index = np.random.randint(len(X_test))
    word_label = words[results[index]]
    plt.plot()
    plt.title(word_label)
    plt.imshow(X_test[index])
    plt.show()

In [ ]:
# This saves an HTML version of the notebook for later viewing
# NOTE: Make sure to check if the html was completely saved, sometimes it doesn't
from nbconvert import HTMLExporter
import codecs
import nbformat
exporter = HTMLExporter()
output_notebook = nbformat.read("Training.ipynb", as_version=nbformat.NO_CONVERT)
output, resources = exporter.from_notebook_node(output_notebook)
codecs.open(model_prefix + ".html", "w").write(output)